In [1]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

In [2]:
# empl = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/employees.csv') # MAC
empl = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\employees.csv') # WIN

In [3]:
# depart = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/departments.csv') # MAC
depart = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\departments.csv') # WIN

In [7]:
# events = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/events.csv') # MAC
events = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\events.csv') # WIN

In [ ]:
# events = pd.read_csv('/Users/vladislavlipkin/Documents/PycharmProjects/notes/temp2_project/time_tracking.csv') # MAC
time_track = pd.read_csv(r'C:\Users\Incognitus\PycharmProjects\notes_git\temp2_project\time_tracking.csv') # WIN

In [14]:
# 2 version

import csv
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('ru_RU')

# --- Departments ---
departments = [
    {"id_department": 1, "name_department": "Связисты"},
    {"id_department": 2, "name_department": "Техподдержка"},
    {"id_department": 3, "name_department": "Информационная безопасность"},
    {"id_department": 4, "name_department": "Инфраструктурные системы"},
    {"id_department": 5, "name_department": "Сопровождение прикладных систем"},
    {"id_department": 6, "name_department": "Кабельщики"}
]

# Генерация head_id и budget
for dep in departments:
    dep["head_id"] = None  # можно потом раскидать по сотрудникам
    dep["budget"] = random.randint(1_000_000, 5_000_000)


# Кол-во сотрудников по отделам
department_sizes = {
    1: 8,
    2: 14,
    3: 8,
    4: 6,
    5: 10,
    6: 8
}

# --- Positions by department ---
positions = {
    1: ["Инженер связи", "Специалист по телефонии", "Техник связи", "Системный инженер АТС"],
    2: ["Инженер технической поддержки", "Системный администратор", "Специалист helpdesk", "Техник по обслуживанию ПК"],
    3: ["Аналитик ИБ", "Инженер по ИБ", "Специалист SOC", "Администратор средств защиты информации"],
    4: ["Системный администратор", "Инженер серверных систем", "Администратор баз данных", "Специалист по виртуализации"],
    5: ["Системный аналитик", "Специалист по сопровождению 1С", "Администратор приложений", "Инженер по интеграции"],
    6: ["Кабельщик-сварщик", "Монтажник СКС", "Техник по обслуживанию сетей", "Специалист по прокладке кабеля"]
}

# --- Ticket templates (расширенные для 2–5 департаментов) ---
ticket_templates = {
    1: [("Не работает телефон", "Заменен кабель, восстановлена связь"),
        ("Нет сигнала в розетке", "Перепатчена розетка на коммутаторе"),
        ("Шумы в телефоне", "Заменена гарнитура"),
        ("Не проходит вызов", "Проверен шлюз, ошибка устранена"),
        ("Не работает внутренний номер", "Переименован в АТС, восстановлен доступ")],
    2: [("Не подключается сетевой диск", "Подключение восстановлено"),
        ("Не обновляется антивирус", "База обновлена вручную"),
        ("Завис компьютер", "Сбой устранен, диск проверен"),
        ("Не работает Teams", "Переустановлено приложение"),
        ("Ошибка печати", "Установлен новый драйвер принтера"),
        ("Сбой Outlook", "Исправлен профиль почты"),
        ("Не открывается браузер", "Очищен кеш и переустановлено приложение"),
        ("Монитор не включается", "Заменен кабель питания"),
        ("Ошибка учетной записи", "Аккаунт разблокирован"),
        ("Не работает мышь", "Заменено устройство"),
        ("Долгая загрузка", "Отключены автозагрузки"),
        ("Zoom не запускается", "Обновлена версия клиента"),
        ("ПК перегревается", "Проведена чистка от пыли"),
        ("Не работает принтер по Wi-Fi", "Настроено сетевое подключение"),
        ("Ошибка синхронизации OneDrive", "Перезапущен клиент"),
        ("Не работает камера в Zoom", "Обновлены драйверы"),
        ("Синий экран Windows", "Выполнено обновление драйверов"),
        ("USB устройство не определяется", "Переустановлены драйверы контроллера"),
        ("Неверное время на компьютере", "Синхронизированы часы через NTP"),
        ("Не подключается Wi-Fi", "Сброшены настройки адаптера"),
        ("Тормозит браузер", "Очищен кеш, отключены расширения"),
        ("Проблемы с VPN", "Заново настроен клиент"),
        ("Автоматически перезагружается ПК", "Проверена система охлаждения"),
        ("Проблемы с клавиатурой", "Заменено устройство"),
        ("Некорректная работа Excel", "Переустановлен Office"),
        ("Не открываются PDF файлы", "Установлен новый ридер")],
    3: [("Вход с неизвестного устройства", "Доступ временно заблокирован"),
        ("Попытка подбора пароля", "IP заблокирован"),
        ("Нарушение политики паролей", "Сотрудник проинструктирован"),
        ("Обнаружен троян", "ОС переустановлена"),
        ("Не прошёл аудит логов", "Настроено централизованное логирование"),
        ("Подозрение на утечку", "Проведена проверка каналов"),
        ("VPN не авторизуется", "Заменены ключи доступа"),
        ("Нарушение прав доступа", "Исправлены ACL"),
        ("Подозрительная активность", "Учётная запись заблокирована"),
        ("Несанкционированное ПО", "Программа удалена"),
        ("Сбой обновления антивируса", "Ручное обновление базы"),
        ("Фишинговая рассылка", "Домен добавлен в blacklist"),
        ("Обнаружена подозрительная программа", "Удалено и отправлено на анализ"),
        ("Несанкционированное подключение к Wi-Fi", "Сеть заблокирована"),
        ("Утечка данных через почту", "Отправка ограничена, проведено расследование"),
        ("Массовая рассылка спама с внутреннего ящика", "Учетка заблокирована"),
        ("Обнаружен руткит", "Система переустановлена"),
        ("Нарушение разграничения доступа", "Исправлены роли пользователей"),
        ("Атака на веб-приложение", "Включена WAF защита"),
        ("Брутфорс учетных записей", "Введен captcha"),
        ("Срабатывание DLP", "Доступ ограничен"),
        ("Обнаружен кейлоггер", "Устройство изолировано"),
        ("Удаленное подключение без согласования", "Сессия завершена, инцидент зафиксирован"),
        ("Сертификат безопасности просрочен", "Выпущен новый"),
        ("Аномальная активность в журнале", "Выполнена проверка событий")],
    4: [("Сбой контроллера домена", "Сервер перезагружен, доступ восстановлен"),
        ("DNS не отвечает", "Служба перезапущена"),
        ("DHCP раздаёт неверные IP", "Исправлен пул адресов"),
        ("Не проходит логин", "Синхронизирован каталог AD"),
        ("Медленный файловый сервер", "Перераспределены ресурсы"),
        ("Сбой SAN", "Заменен неисправный диск"),
        ("Почтовый сервер не отвечает", "Перезапущен сервис"),
        ("Резервное копирование не проходит", "Расширен диск бэкап-сервера"),
        ("Сбой гипервизора", "ВМ перенесены"),
        ("Ошибка в кластере", "Восстановлена нода"),
        ("VPN нестабилен", "Обновлены сертификаты"),
        ("Мониторинг не работал", "Агент перезапущен"),
        ("Нехватка места на сервере", "Расширен диск"),
        ("Сбой RAID-массива", "Восстановлен из резерва"),
        ("Проблемы с репликацией AD", "Восстановлена синхронизация"),
        ("Сервер зависает", "Установлены обновления"),
        ("Ошибка в системе виртуализации", "Перезапущен сервис"),
        ("Медленный доступ к БД", "Оптимизированы запросы"),
        ("Ошибка лицензирования", "Добавлены новые лицензии"),
        ("Сбой в балансировщике нагрузки", "Перезапущен сервис"),
        ("Аппаратный сбой сервера", "Заменен блок питания"),
        ("Не работают групповые политики", "Исправлены объекты GPO"),
        ("Проблемы с репликацией DFS", "Проведена синхронизация"),
        ("Не запускается служба Exchange", "Служба перезапущена")],
    5: [("Ошибка выгрузки в Excel", "Исправлена обработка"),
        ("Ошибка расчета зарплаты", "Обновлен регламент"),
        ("Ошибка при счете", "Исправлена форма документа"),
        ("Некорректный курс валюты", "Данные обновлены"),
        ("CRM не отправляет письма", "SMTP проверен, доступ восстановлен"),
        ("Не формируется отчет", "Переписан запрос"),
        ("Ошибка авторизации", "Обновлены роли"),
        ("Зависание при выгрузке", "Оптимизирована интеграция"),
        ("Некорректные данные", "Исправлены справочники"),
        ("Ошибка ЭДО", "Установлен сертификат"),
        ("Не отображаются товары", "Исправлен обмен"),
        ("Задвоение заказов", "Исправлен бизнес-процесс"),
        ("Неверные права в CRM", "Изменены ACL"),
        ("Ошибка при закрытии месяца", "Обновлен модуль"),
        ("Некорректный расчет налогов", "Исправлен алгоритм"),
        ("Не загружаются курсы валют", "Настроен обмен"),
        ("Ошибка в отчетности", "Обновлен шаблон"),
        ("Не работает интеграция с банком", "Исправлен коннектор"),
        ("Задержка при формировании отчета", "Оптимизирован запрос"),
        ("Проблемы с выгрузкой в XML", "Исправлен формат"),
        ("Не отображаются графики", "Добавлена библиотека"),
        ("Некорректная авторизация в CRM", "Исправлена схема SSO"),
        ("Ошибка импорта данных", "Исправлен парсер"),
        ("Не отправляются уведомления", "Исправлена очередь сообщений"),
        ("Проблемы с ERP", "Перезапущен сервис"),
        ("Некорректный расчет себестоимости", "Исправлена формула"),
        ("Ошибка обмена с маркетплейсом", "Исправлен модуль интеграции"),
        ("Сбой при создании документа", "Исправлена форма")],
    6: [("Не работает интернет в кабинете", "Заменен сетевой кабель"),
        ("Оборван кабель в шахте", "Проведена новая линия"),
        ("Нет связи на этаже", "Подключен резервный кабель"),
        ("Кабель поврежден", "Произведен ремонт"),
        ("Плохой сигнал", "Заменен патч-корд")]
}

# --- Generate Employees ---
employees = []
emp_id = 1
for dep in departments:
    dep_id = dep["id_department"]
    dep_name = dep["name_department"]
    for _ in range(department_sizes[dep_id]):
        hire_date = fake.date_between(start_date="-5y", end_date="today")
        termination_date = None if random.random() > 0.1 else fake.date_between(start_date=hire_date, end_date="today")
        salary = random.randint(40000, 120000)
        full_name = fake.name()
        
        employees.append({
            "id": emp_id,
            "id_department": dep_id,
            "full_name": full_name,
            "phone_number": fake.msisdn()[:11],
            "email": fake.email(),
            "position": random.choice(positions[dep_id]),
            "is_active": termination_date is None,
            "hire_date": hire_date,
            "termination_date": termination_date,
            "salary": salary,
            "manager_id": None
        })
        emp_id += 1

# --- Generate pool of reporters (один раз для всей базы) ---
reporters = [fake.name() for _ in range(1500)]

# --- Generate Events and time_tracking---
events = []
time_tracking = []

TOTAL_EVENTS = 5000

for event_id in range(1, TOTAL_EVENTS + 1):
    dep = random.choice(departments)
    dep_id = dep["id_department"]
    title, description = random.choice(ticket_templates[dep_id])

    created_at = fake.date_time_between(start_date="-5y", end_date="now")
    deadline = created_at + timedelta(days=random.randint(1, 14))

    status = None
    completed_at = None
    closed_at = None

    if random.random() < 0.1:
        status = "в работе"
    else:
        completed_at = created_at + timedelta(hours=random.randint(2, 24*14))
        closed_at = completed_at + timedelta(hours=random.randint(1, 72))
        if random.random() < 0.1:
            status = "отклонена"
        else:
            if completed_at <= deadline:
                status = "выполнена"
            else:
                status = "просроченная"

    priority = random.choice(["низкий", "средний", "высокий"])

    # --- Events без дат ---
    events.append({
        "id_event": event_id,
        "title": title,
        "description": description,
        "status": status,
        "priority": priority,
        "assigned_to": random.choice(employees)["id"],
        "reported_by": random.choice(reporters),
        "id_department": dep_id
    })

    # --- Time tracking с датами ---
    time_tracking.append({
        "id_event": event_id,
        "created_at": created_at,
        "deadline": deadline,
        "completed_at": completed_at,
        "closed_at": closed_at
    })

# --- Save CSV ---

with open("departments.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_department", "name_department", "head_id", "budget"])
    for dep in departments:
        writer.writerow([dep["id_department"], dep["name_department"], dep["head_id"], dep["budget"]])

with open("employees.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "id_department", "full_name", "phone_number", "email", "position", "is_active", "hire_date", "termination_date", "salary", "manager_id"])
    for e in employees:
        writer.writerow([e["id"], e["id_department"], e["full_name"], e["phone_number"], e["email"], e["position"], e["is_active"], e["hire_date"], e["termination_date"], e["salary"], e["manager_id"]])

with open("events.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_event", "title", "description", "status", "priority", "assigned_to", "reported_by", "id_department"])
    for ev in events:
        writer.writerow([ev["id_event"], ev["title"], ev["description"], ev["status"], ev["priority"], ev["assigned_to"], ev["reported_by"], ev["id_department"]])

with open("time_tracking.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_event", "created_at", "deadline", "completed_at", "closed_at"])
    for t in time_tracking:
        writer.writerow([t["id_event"], t["created_at"], t["deadline"], t["completed_at"], t["closed_at"]])


In [1]:
# 3 version

import csv
import random
from faker import Faker
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

fake = Faker('ru_RU')

# --- Departments ---
departments = [
    {"id_department": 1, "name_department": "Связисты"},
    {"id_department": 2, "name_department": "Техподдержка"},
    {"id_department": 3, "name_department": "Информационная безопасность"},
    {"id_department": 4, "name_department": "Инфраструктурные системы"},
    {"id_department": 5, "name_department": "Сопровождение прикладных систем"},
    {"id_department": 6, "name_department": "Кабельщики"}
]

# Генерация head_id и budget
for dep in departments:
    dep["head_id"] = None  # можно потом раскидать по сотрудникам
    dep["budget"] = random.randint(1_000_000, 5_000_000)


# Кол-во сотрудников по отделам
department_sizes = {
    1: 8,
    2: 14,
    3: 8,
    4: 6,
    5: 10,
    6: 8
}

# --- Positions by department ---
positions = {
    1: ["Инженер связи", "Специалист по телефонии", "Техник связи", "Системный инженер АТС"],
    2: ["Инженер технической поддержки", "Системный администратор", "Специалист helpdesk", "Техник по обслуживанию ПК"],
    3: ["Аналитик ИБ", "Инженер по ИБ", "Специалист SOC", "Администратор средств защиты информации"],
    4: ["Системный администратор", "Инженер серверных систем", "Администратор баз данных", "Специалист по виртуализации"],
    5: ["Системный аналитик", "Специалист по сопровождению 1С", "Администратор приложений", "Инженер по интеграции"],
    6: ["Кабельщик-сварщик", "Монтажник СКС", "Техник по обслуживанию сетей", "Специалист по прокладке кабеля"]
}

# --- Salary ranges by position (грейды) ---
salary_ranges = {
    # Отдел 1: Связисты
    "Инженер связи": (60000, 90000),
    "Специалист по телефонии": (50000, 80000),
    "Техник связи": (40000, 60000),
    "Системный инженер АТС": (70000, 110000),

    # Отдел 2: Техподдержка
    "Инженер технической поддержки": (50000, 80000),
    "Системный администратор": (70000, 120000),
    "Специалист helpdesk": (40000, 65000),
    "Техник по обслуживанию ПК": (40000, 60000),

    # Отдел 3: Инфобез
    "Аналитик ИБ": (70000, 110000),
    "Инженер по ИБ": (80000, 120000),
    "Специалист SOC": (70000, 100000),
    "Администратор средств защиты информации": (75000, 115000),

    # Отдел 4: Инфраструктура
    "Системный администратор": (70000, 120000),
    "Инженер серверных систем": (80000, 120000),
    "Администратор баз данных": (90000, 130000),
    "Специалист по виртуализации": (85000, 125000),

    # Отдел 5: Прикладные системы
    "Системный аналитик": (80000, 120000),
    "Специалист по сопровождению 1С": (70000, 100000),
    "Администратор приложений": (75000, 110000),
    "Инженер по интеграции": (85000, 125000),

    # Отдел 6: Кабельщики
    "Кабельщик-сварщик": (40000, 60000),
    "Монтажник СКС": (45000, 65000),
    "Техник по обслуживанию сетей": (50000, 70000),
    "Специалист по прокладке кабеля": (45000, 65000),

    # Руководители
    "Руководитель отдела": (110000, 160000)
}

# --- Ticket templates (расширенные для 2–5 департаментов) ---
ticket_templates = {
    1: [("Не работает телефон", "Заменен кабель, восстановлена связь"),
        ("Нет сигнала в розетке", "Перепатчена розетка на коммутаторе"),
        ("Шумы в телефоне", "Заменена гарнитура"),
        ("Не проходит вызов", "Проверен шлюз, ошибка устранена"),
        ("Не работает внутренний номер", "Переименован в АТС, восстановлен доступ")],
    2: [("Не подключается сетевой диск", "Подключение восстановлено"),
        ("Не обновляется антивирус", "База обновлена вручную"),
        ("Завис компьютер", "Сбой устранен, диск проверен"),
        ("Не работает Teams", "Переустановлено приложение"),
        ("Ошибка печати", "Установлен новый драйвер принтера"),
        ("Сбой Outlook", "Исправлен профиль почты"),
        ("Не открывается браузер", "Очищен кеш и переустановлено приложение"),
        ("Монитор не включается", "Заменен кабель питания"),
        ("Ошибка учетной записи", "Аккаунт разблокирован"),
        ("Не работает мышь", "Заменено устройство"),
        ("Долгая загрузка", "Отключены автозагрузки"),
        ("Zoom не запускается", "Обновлена версия клиента"),
        ("ПК перегревается", "Проведена чистка от пыли"),
        ("Не работает принтер по Wi-Fi", "Настроено сетевое подключение"),
        ("Ошибка синхронизации OneDrive", "Перезапущен клиент"),
        ("Не работает камера в Zoom", "Обновлены драйверы"),
        ("Синий экран Windows", "Выполнено обновление драйверов"),
        ("USB устройство не определяется", "Переустановлены драйверы контроллера"),
        ("Неверное время на компьютере", "Синхронизированы часы через NTP"),
        ("Не подключается Wi-Fi", "Сброшены настройки адаптера"),
        ("Тормозит браузер", "Очищен кеш, отключены расширения"),
        ("Проблемы с VPN", "Заново настроен клиент"),
        ("Автоматически перезагружается ПК", "Проверена система охлаждения"),
        ("Проблемы с клавиатурой", "Заменено устройство"),
        ("Некорректная работа Excel", "Переустановлен Office"),
        ("Не открываются PDF файлы", "Установлен новый ридер")],
    3: [("Вход с неизвестного устройства", "Доступ временно заблокирован"),
        ("Попытка подбора пароля", "IP заблокирован"),
        ("Нарушение политики паролей", "Сотрудник проинструктирован"),
        ("Обнаружен троян", "ОС переустановлена"),
        ("Не прошёл аудит логов", "Настроено централизованное логирование"),
        ("Подозрение на утечку", "Проведена проверка каналов"),
        ("VPN не авторизуется", "Заменены ключи доступа"),
        ("Нарушение прав доступа", "Исправлены ACL"),
        ("Подозрительная активность", "Учётная запись заблокирована"),
        ("Несанкционированное ПО", "Программа удалена"),
        ("Сбой обновления антивируса", "Ручное обновление базы"),
        ("Фишинговая рассылка", "Домен добавлен в blacklist"),
        ("Обнаружена подозрительная программа", "Удалено и отправлено на анализ"),
        ("Несанкционированное подключение к Wi-Fi", "Сеть заблокирована"),
        ("Утечка данных через почту", "Отправка ограничена, проведено расследование"),
        ("Массовая рассылка спама с внутреннего ящика", "Учетка заблокирована"),
        ("Обнаружен руткит", "Система переустановлена"),
        ("Нарушение разграничения доступа", "Исправлены роли пользователей"),
        ("Атака на веб-приложение", "Включена WAF защита"),
        ("Брутфорс учетных записей", "Введен captcha"),
        ("Срабатывание DLP", "Доступ ограничен"),
        ("Обнаружен кейлоггер", "Устройство изолировано"),
        ("Удаленное подключение без согласования", "Сессия завершена, инцидент зафиксирован"),
        ("Сертификат безопасности просрочен", "Выпущен новый"),
        ("Аномальная активность в журнале", "Выполнена проверка событий")],
    4: [("Сбой контроллера домена", "Сервер перезагружен, доступ восстановлен"),
        ("DNS не отвечает", "Служба перезапущена"),
        ("DHCP раздаёт неверные IP", "Исправлен пул адресов"),
        ("Не проходит логин", "Синхронизирован каталог AD"),
        ("Медленный файловый сервер", "Перераспределены ресурсы"),
        ("Сбой SAN", "Заменен неисправный диск"),
        ("Почтовый сервер не отвечает", "Перезапущен сервис"),
        ("Резервное копирование не проходит", "Расширен диск бэкап-сервера"),
        ("Сбой гипервизора", "ВМ перенесены"),
        ("Ошибка в кластере", "Восстановлена нода"),
        ("VPN нестабилен", "Обновлены сертификаты"),
        ("Мониторинг не работал", "Агент перезапущен"),
        ("Нехватка места на сервере", "Расширен диск"),
        ("Сбой RAID-массива", "Восстановлен из резерва"),
        ("Проблемы с репликацией AD", "Восстановлена синхронизация"),
        ("Сервер зависает", "Установлены обновления"),
        ("Ошибка в системе виртуализации", "Перезапущен сервис"),
        ("Медленный доступ к БД", "Оптимизированы запросы"),
        ("Ошибка лицензирования", "Добавлены новые лицензии"),
        ("Сбой в балансировщике нагрузки", "Перезапущен сервис"),
        ("Аппаратный сбой сервера", "Заменен блок питания"),
        ("Не работают групповые политики", "Исправлены объекты GPO"),
        ("Проблемы с репликацией DFS", "Проведена синхронизация"),
        ("Не запускается служба Exchange", "Служба перезапущена")],
    5: [("Ошибка выгрузки в Excel", "Исправлена обработка"),
        ("Ошибка расчета зарплаты", "Обновлен регламент"),
        ("Ошибка при счете", "Исправлена форма документа"),
        ("Некорректный курс валюты", "Данные обновлены"),
        ("CRM не отправляет письма", "SMTP проверен, доступ восстановлен"),
        ("Не формируется отчет", "Переписан запрос"),
        ("Ошибка авторизации", "Обновлены роли"),
        ("Зависание при выгрузке", "Оптимизирована интеграция"),
        ("Некорректные данные", "Исправлены справочники"),
        ("Ошибка ЭДО", "Установлен сертификат"),
        ("Не отображаются товары", "Исправлен обмен"),
        ("Задвоение заказов", "Исправлен бизнес-процесс"),
        ("Неверные права в CRM", "Изменены ACL"),
        ("Ошибка при закрытии месяца", "Обновлен модуль"),
        ("Некорректный расчет налогов", "Исправлен алгоритм"),
        ("Не загружаются курсы валют", "Настроен обмен"),
        ("Ошибка в отчетности", "Обновлен шаблон"),
        ("Не работает интеграция с банком", "Исправлен коннектор"),
        ("Задержка при формировании отчета", "Оптимизирован запрос"),
        ("Проблемы с выгрузкой в XML", "Исправлен формат"),
        ("Не отображаются графики", "Добавлена библиотека"),
        ("Некорректная авторизация в CRM", "Исправлена схема SSO"),
        ("Ошибка импорта данных", "Исправлен парсер"),
        ("Не отправляются уведомления", "Исправлена очередь сообщений"),
        ("Проблемы с ERP", "Перезапущен сервис"),
        ("Некорректный расчет себестоимости", "Исправлена формула"),
        ("Ошибка обмена с маркетплейсом", "Исправлен модуль интеграции"),
        ("Сбой при создании документа", "Исправлена форма")],
    6: [("Не работает интернет в кабинете", "Заменен сетевой кабель"),
        ("Оборван кабель в шахте", "Проведена новая линия"),
        ("Нет связи на этаже", "Подключен резервный кабель"),
        ("Кабель поврежден", "Произведен ремонт"),
        ("Плохой сигнал", "Заменен патч-корд")]
}

# --- Generate Services ---
services = []
service_id = 1

# Карта подкатегорий по отделам
subcategory_map = {
    1: ["Телефония", "Кабельная разводка", "АТС"],
    2: ["ОС/Сеть", "ПО", "Периферия"],
    3: ["Вирусы", "Политики доступа", "Подозрительная активность"],
    4: ["AD/DNS/DHCP", "Файловые серверы", "Виртуализация", "Почтовая система"],
    5: ["1С/ERP", "CRM", "Интеграции"],
    6: ["Прокладка", "Повреждения", "Сигнал"]
}

# Генерация услуг (по категориям и подкатегориям)
for dep_id, subcats in subcategory_map.items():
    category = next(d["name_department"] for d in departments if d["id_department"] == dep_id)
    for subcat in subcats:
        default_cost = random.randint(2000, 10000)
        services.append({
            "id_service": service_id,
            "category": category,
            "subcategory": subcat,
            "default_cost": default_cost
        })
        service_id += 1


# --- Generate Employees ---
employees = []
emp_id = 1
for dep in departments:
    dep_id = dep["id_department"]
    dep_name = dep["name_department"]
    for _ in range(department_sizes[dep_id]):
        hire_date = fake.date_between(start_date="-5y", end_date="today")
        termination_date = None if random.random() > 0.1 else fake.date_between(start_date=hire_date, end_date="today")
        position = random.choice(positions[dep_id])
        salary_min, salary_max = salary_ranges.get(position, (40000, 120000))
        salary = random.randint(salary_min, salary_max)
        full_name = fake.name()
        
        employees.append({
            "id": emp_id,
            "id_department": dep_id,
            "full_name": full_name,
            "phone_number": fake.msisdn()[:11],
            "email": fake.email(),
            "position": random.choice(positions[dep_id]),
            "is_active": termination_date is None,
            "hire_date": hire_date,
            "termination_date": termination_date,
            "salary": salary,
            "manager_id": None
        })
        emp_id += 1


# --- Generate expenses ---
finance_expenses = []
expense_id = 1

# --- Зарплаты сотрудников ---
for emp in employees:
    dep_id = emp["id_department"]
    salary = emp["salary"]

    # за последние 60 месяцев (5 лет)
    for month in range(60):
        base_date = datetime.now() - relativedelta(months=month)

        # Зарплата 10 числа (60%)
        finance_expenses.append({
            "id_expense": expense_id,
            "id_department": dep_id,
            "expense_type": "зарплата",
            "amount": salary * 0.6,
            "date": base_date.replace(day=10)
        })
        expense_id += 1

        # Аванс 25 числа (40%)
        finance_expenses.append({
            "id_expense": expense_id,
            "id_department": dep_id,
            "expense_type": "аванс",
            "amount": salary * 0.4,
            "date": base_date.replace(day=25)
        })
        expense_id += 1


# --- Операционные расходы ---
expense_types = [
    "аренда ПО", "закупка оборудования", "интернет", "обучение персонала",
    "конференции", "канцелярия", "связь", "электричество",
    "ремонт техники", "командировки"
]

for dep in departments:
    dep_id = dep["id_department"]

    # теперь делаем расходы "регулярнее" (например, каждый месяц 1-3 случайных)
    for month in range(60):
        base_date = datetime.now() - relativedelta(months=month)
        for _ in range(random.randint(1, 3)):
            expense_type = random.choice(expense_types)
            amount = random.randint(5000, 200000)
            day = random.randint(1, 28)  # чтобы не вылетало по датам

            finance_expenses.append({
                "id_expense": expense_id,
                "id_department": dep_id,
                "expense_type": expense_type,
                "amount": amount,
                "date": base_date.replace(day=day)
            })
            expense_id += 1


# --- Перемешиваем все расходы по дате ---
finance_expenses.sort(key=lambda x: x["date"])



# --- Назначаем руководителей отделов ---
for dep in departments:
    dep_id = dep["id_department"]
    dept_employees = [e for e in employees if e["id_department"] == dep_id]
    head = random.choice(dept_employees)
    
    dep["head_id"] = head["id"]
    head["position"] = "Руководитель отдела"
    
    # пересчёт зарплаты для руководителя по грейду
    salary_min, salary_max = salary_ranges["Руководитель отдела"]
    head["salary"] = random.randint(salary_min, salary_max)
    
    for e in dept_employees:
        if e["id"] != head["id"]:
            e["manager_id"] = head["id"]


# --- Generate pool of reporters (один раз для всей базы) ---
reporters = [fake.name() for _ in range(1500)]

# --- Generate Events and time_tracking---
events = []
time_tracking = []

TOTAL_EVENTS = 5000

reject_reasons = [
    "Проблема устранилась сама",
    "Пользователь отсутствует на рабочем месте",
    "Пользователь в отпуске",
    "Заявка дублирует другую",
    "Заявка отменена инициатором",
    "Нет необходимых данных для выполнения",
    " "
]

for event_id in range(1, TOTAL_EVENTS + 1):
    dep = random.choice(departments)
    dep_id = dep["id_department"]

    # Выбираем случайную услугу для данного отдела
    category = next(d["name_department"] for d in departments if d["id_department"] == dep_id)
    service = random.choice([s for s in services if s["category"] == category])
    id_service = service["id_service"]

    # Берём шаблон тикета
    title, description = random.choice(ticket_templates[dep_id])

    created_at = fake.date_time_between(start_date="-5y", end_date="now")
    deadline = created_at + timedelta(days=random.randint(1, 14))

    status = None
    completed_at = None
    closed_at = None

    if random.random() < 0.1:
        status = "в работе"
    else:
        completed_at = created_at + timedelta(hours=random.randint(2, 24*14))
        closed_at = completed_at + timedelta(hours=random.randint(1, 72))
        if random.random() < 0.1:
            status = "отклонена"
            description = random.choice(reject_reasons)  # <-- вот тут причина идёт в description
        else:
            if completed_at <= deadline:
                status = "выполнена"
            else:
                status = "просроченная"

    priority = random.choice(["низкий", "средний", "высокий"])

    # --- Events ---
    events.append({
        "id_event": event_id,
        "title": title,
        "description": description,  # <-- тут может быть либо нормальное описание, либо причина отклонения
        "status": status,
        "priority": priority,
        "assigned_to": random.choice(employees)["id"],
        "reported_by": random.choice(reporters),
        "id_department": dep_id,
        "id_service": id_service
    })

    # --- Time tracking с датами ---
    time_tracking.append({
        "id_event": event_id,
        "created_at": created_at,
        "deadline": deadline,
        "completed_at": completed_at,
        "closed_at": closed_at
    })




# --- Save CSV ---

with open("departments.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_department", "name_department", "head_id", "budget"])
    for dep in departments:
        writer.writerow([dep["id_department"], dep["name_department"], dep["head_id"], dep["budget"]])

with open("employees.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "id_department", "full_name", "phone_number", "email", "position",
                     "is_active", "hire_date", "termination_date", "salary", "manager_id"])
    for e in employees:
        writer.writerow([e["id"], e["id_department"], e["full_name"], e["phone_number"], e["email"],
                         e["position"], e["is_active"], e["hire_date"], e["termination_date"],
                         e["salary"], e["manager_id"]])

with open("events.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_event", "title", "description", "status", "priority",
                     "assigned_to", "reported_by", "id_department", "id_service"])
    for ev in events:
        writer.writerow([ev["id_event"], ev["title"], ev["description"], ev["status"], ev["priority"],
                         ev["assigned_to"], ev["reported_by"], ev["id_department"], ev["id_service"]])

with open("time_tracking.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_event", "created_at", "deadline", "completed_at", "closed_at"])
    for t in time_tracking:
        writer.writerow([t["id_event"], t["created_at"], t["deadline"], t["completed_at"], t["closed_at"]])

with open("services.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_service", "category", "subcategory", "default_cost"])
    for s in services:
        writer.writerow([s["id_service"], s["category"], s["subcategory"], s["default_cost"]])

with open("finance_expenses.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["id_expense", "id_department", "expense_type", "amount", "date"])
    writer.writeheader()
    writer.writerows(finance_expenses)

In [ ]:
# обратить внимание на таблицу finance expenses так как там непонятно зарплаты и затраты 